In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [2]:

#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [3]:
df_ppd.head(5)

,{726BF13A-993E-0A46-E053-6C04A8C01D0D},75000,2018-06-20 00:00,DL13 4BA,T,N,F,21,Unnamed: 8,HIGHFIELDS,TOW LAW,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM.1,A,A.1
0,{726BF13A-993F-0A46-E053-6C04A8C01D0D},115000,2018-07-13 00:00,DL17 9LB,S,N,F,4,NaN,THE LANE,WEST CORNFORTH,FERRYHILL,COUNTY DURHAM,COUNTY DURHAM,A,A
1,{726BF13A-9940-0A46-E053-6C04A8C01D0D},24000,2018-04-10 00:00,SR7 9AG,F,N,L,20B,NaN,WOODS TERRACE,MURTON,SEAHAM,COUNTY DURHAM,COUNTY DURHAM,A,A
2,{726BF13A-9941-0A46-E053-6C04A8C01D0D},56000,2018-06-22 00:00,DL5 5PS,T,N,F,6,NaN,HEILD CLOSE,NaN,NEWTON AYCLIFFE,COUNTY DURHAM,COUNTY DURHAM,A,A
3,{726BF13A-9942-0A46-E053-6C04A8C01D0D},220000,2018-05-25 00:00,DL16 7HE,D,N,F,25,NaN,BECKWITH CLOSE,KIRK MERRINGTON,SPENNYMOOR,COUNTY DURHAM,COUNTY DURHAM,A,A
4,{726BF13A-9943-0A46-E053-6C04A8C01D0D},58000,2018-05-09 00:00,DL14 6FH,F,N,L,23,NaN,AINTREE DRIVE,NaN,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM,A,A


In [4]:
df_ppd.shape

(993670, 16)

In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [6]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [7]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [8]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [9]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [10]:
# Display the dataframe
df_affordable

,Street,Avg_Price
194,ALBION SQUARE,2.450000e+06
388,ANHALT ROAD,2.435000e+06
402,ANSDELL TERRACE,2.250000e+06
417,APPLEGARTH ROAD,2.400000e+06
845,BARONSMEAD ROAD,2.375000e+06
969,BEAUCLERC ROAD,2.480000e+06
1091,BELVEDERE DRIVE,2.340000e+06
1204,BICKENHALL STREET,2.208500e+06
1242,BIRCHLANDS AVENUE,2.217000e+06
1539,BRAMPTON GROVE,2.456875e+06


In [11]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [12]:

for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 194
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 194, dtype: object
item.Street only: ALBION SQUARE
index: 388
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 388, dtype: object
item.Street only: ANHALT ROAD
index: 402
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 402, dtype: object
item.Street only: ANSDELL TERRACE
index: 417
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 417, dtype: object
item.Street only: APPLEGARTH ROAD
index: 845
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 845, dtype: object
item.Street only: BARONSMEAD ROAD
index: 969
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 969, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1091
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1091, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1204
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [13]:
geolocator = Nominatim()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [14]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_affordable

,Street,Avg_Price,city_coord
194,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
388,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
402,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
417,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)"
845,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
969,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1091,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)"
1204,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)"
1242,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"
1539,BRAMPTON GROVE,2.456875e+06,"(51.5703648, -0.2833944)"


In [16]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [17]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
194,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
388,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
402,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
417,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)",53.749244,-0.326780
845,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
969,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1091,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)",38.201316,-84.623076
1204,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1242,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1539,BRAMPTON GROVE,2.456875e+06,"(51.5703648, -0.2833944)",51.570365,-0.283394


In [18]:
df = df_affordable.drop(columns=['city_coord'])

In [19]:
df

,Street,Avg_Price,Latitude,Longitude
194,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
388,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
402,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
417,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
845,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
969,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1091,BELVEDERE DRIVE,2.340000e+06,38.201316,-84.623076
1204,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934
1242,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1539,BRAMPTON GROVE,2.456875e+06,51.570365,-0.283394


In [20]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [21]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [22]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3
CLIENT_SECRET:QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALEDONIAN ROAD
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HOXTON SQUARE
HUNTER ROAD
JACKSON

In [28]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
4,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
5,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
6,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
7,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant
8,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
9,ALBION SQUARE,-41.273758,173.289393,cod and lobster,-41.275203,173.283747,Seafood Restaurant


In [29]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,28,28,28,28,28,28
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,56,56,56,56,56,56
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,15,15,15,15,15,15
BEAUCLERC ROAD,28,28,28,28,28,28
BICKENHALL STREET,100,100,100,100,100,100
BIRCHLANDS AVENUE,12,12,12,12,12,12
BRAMPTON GROVE,2,2,2,2,2,2


In [30]:

# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 348 uniques categories.


In [31]:
location_venues.shape

(5495, 7)

In [32]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.035714,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,ANHALT ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,ANSDELL TERRACE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.017857,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.017857,0.000000,0.000000
3,APPLEGARTH ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,BARONSMEAD ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,BEAUCLERC ROAD,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,BICKENHALL STREET,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.010000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.010000,0.000000,0.010000,0.010000,0.000000
7,BIRCHLANDS AVENUE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,BRAMPTON GROVE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,BRIARDALE GARDENS,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
london_grouped.shape

(141, 349)

In [35]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.21
1  Indian Restaurant  0.11
2         Restaurant  0.07
3                Pub  0.07
4                Bar  0.07


----ANHALT ROAD----
                  venue  freq
0                   Pub  0.27
1         Grocery Store  0.13
2                 Diner  0.07
3                Garden  0.07
4  Gym / Fitness Center  0.07


----ANSDELL TERRACE----
                venue  freq
0  Italian Restaurant  0.07
1      Clothing Store  0.07
2                 Pub  0.05
3               Hotel  0.05
4   Indian Restaurant  0.05


----APPLEGARTH ROAD----
               venue  freq
0             Casino  0.25
1          Nightclub  0.25
2                Bar  0.25
3                Pub  0.25
4  Accessories Store  0.00


----BARONSMEAD ROAD----
               venue  freq
0     Breakfast Spot  0.07
1                Pub  0.07
2  Food & Drink Shop  0.07
3        Coffee Shop  0.07
4          Bookstore  0.07


----BEAUCLERC ROAD----
                ven

In [36]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [38]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Indian Restaurant,Bar,Pub,Restaurant,Coffee Shop,Burger Joint,New American Restaurant,French Restaurant,Brewery
1,ANHALT ROAD,Pub,Grocery Store,Japanese Restaurant,Plaza,Gym / Fitness Center,Cocktail Bar,English Restaurant,Pizza Place,Chinese Restaurant,Garden
2,ANSDELL TERRACE,Italian Restaurant,Clothing Store,Pub,Indian Restaurant,Hotel,English Restaurant,Garden,French Restaurant,Café,Juice Bar
3,APPLEGARTH ROAD,Nightclub,Bar,Pub,Casino,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
4,BARONSMEAD ROAD,Pizza Place,Movie Theater,Thai Restaurant,Nature Preserve,Coffee Shop,Bookstore,Community Center,Food & Drink Shop,Park,Breakfast Spot


In [39]:
venues_sorted.shape

(141, 11)

In [40]:
london_grouped.shape

(141, 349)

In [41]:
london_grouped=df

In [42]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 4, 0, 4, 2, 1, 2, 0, 0, 1, 4, 4, 4, 1, 4, 2, 2, 1, 4, 0, 1, 3,
       3, 4, 1, 1, 0, 4, 3, 0, 4, 2, 2, 2, 3, 4, 4, 2, 0, 2, 2, 3, 0, 4,
       0, 3, 0, 0, 4, 2], dtype=int32)

In [43]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
194,ALBION SQUARE,2450000.0,-41.273758,173.289393
388,ANHALT ROAD,2435000.0,51.480326,-0.166761
402,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
417,APPLEGARTH ROAD,2400000.0,53.749244,-0.326780
845,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [44]:
london_grouped_clustering.shape

df.shape

(152, 4)

In [45]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [46]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [47]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
194,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Indian Restaurant,Bar,Pub,Restaurant,Coffee Shop,Burger Joint,New American Restaurant,French Restaurant,Brewery
388,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,4,Pub,Grocery Store,Japanese Restaurant,Plaza,Gym / Fitness Center,Cocktail Bar,English Restaurant,Pizza Place,Chinese Restaurant,Garden
402,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Italian Restaurant,Clothing Store,Pub,Indian Restaurant,Hotel,English Restaurant,Garden,French Restaurant,Café,Juice Bar
417,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780,4,Nightclub,Bar,Pub,Casino,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
845,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Pizza Place,Movie Theater,Thai Restaurant,Nature Preserve,Coffee Shop,Bookstore,Community Center,Food & Drink Shop,Park,Breakfast Spot
969,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,1,Coffee Shop,Pub,Hotel,Thai Restaurant,Chinese Restaurant,Grocery Store,Gastropub,Supermarket,Fish & Chips Shop,Latin American Restaurant
1091,BELVEDERE DRIVE,2.340000e+06,38.201316,-84.623076,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1204,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934,0,Hotel,Café,Chinese Restaurant,Restaurant,Indian Restaurant,Italian Restaurant,Pub,Pizza Place,Gastropub,Sandwich Place
1242,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,French Restaurant,Chinese Restaurant,Coffee Shop,Brewery,Breakfast Spot,Bakery,Lake,Gym / Fitness Center,Train Station
1539,BRAMPTON GROVE,2.456875e+06,51.570365,-0.283394,1,Middle Eastern Restaurant,Lake,Zoo,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food


In [48]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
402,2250000.0,Italian Restaurant,Clothing Store,Pub,Indian Restaurant,Hotel,English Restaurant,Garden,French Restaurant,Café,Juice Bar
1204,2208500.0,Hotel,Café,Chinese Restaurant,Restaurant,Indian Restaurant,Italian Restaurant,Pub,Pizza Place,Gastropub,Sandwich Place
1242,2217000.0,Pub,French Restaurant,Chinese Restaurant,Coffee Shop,Brewery,Breakfast Spot,Bakery,Lake,Gym / Fitness Center,Train Station
2203,2200000.0,Soccer Field,Nightlife Spot,Playground,Zoo,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
2612,2250000.0,Gastropub,Construction & Landscaping,Zoo,Food Stand,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food


In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
194,2450000.0,Café,Indian Restaurant,Bar,Pub,Restaurant,Coffee Shop,Burger Joint,New American Restaurant,French Restaurant,Brewery
969,2480000.0,Coffee Shop,Pub,Hotel,Thai Restaurant,Chinese Restaurant,Grocery Store,Gastropub,Supermarket,Fish & Chips Shop,Latin American Restaurant
1539,2456875.0,Middle Eastern Restaurant,Lake,Zoo,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
1960,2492500.0,English Restaurant,Supermarket,Rental Car Location,American Restaurant,Café,Park,Pub,Gym,Hardware Store,Flower Shop
2114,2461000.0,Sports Club,Spa,Bus Station,Windmill,Soccer Field,Fried Chicken Joint,Friterie,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop


In [51]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
845,2375000.0,Pizza Place,Movie Theater,Thai Restaurant,Nature Preserve,Coffee Shop,Bookstore,Community Center,Food & Drink Shop,Park,Breakfast Spot
1091,2340000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2046,2375000.0,Pub,Bakery,Indian Restaurant,Grocery Store,Park,Sushi Restaurant,Bookstore,Hotel,Yoga Studio,Ice Cream Shop
2107,2379652.7,Pub,Bakery,Grocery Store,Bookstore,Indian Restaurant,Coffee Shop,Ice Cream Shop,Yoga Studio,Sushi Restaurant,Hostel
2914,2367500.0,Hotel,Pub,Coffee Shop,Italian Restaurant,Café,Garden,Bar,Chinese Restaurant,Mediterranean Restaurant,Residential Building (Apartment / Condo)


In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2220,2.300000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2382,2.286679e+06,Café,Bar,Pub,Pizza Place,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Spa,Gourmet Shop,Belgian Restaurant
2660,2.287500e+06,Pub,Brewery,Art Museum,Gift Shop,Gym / Fitness Center,Zoo,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market
3344,2.298000e+06,Hotel,Zoo,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
4241,2.265000e+06,Pub,Zoo,Food & Drink Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop


In [53]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
388,2435000.0,Pub,Grocery Store,Japanese Restaurant,Plaza,Gym / Fitness Center,Cocktail Bar,English Restaurant,Pizza Place,Chinese Restaurant,Garden
417,2400000.0,Nightclub,Bar,Pub,Casino,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
1617,2397132.0,Sporting Goods Shop,Italian Restaurant,Coffee Shop,Indian Restaurant,Grocery Store,Convenience Store,Food,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
1781,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1896,2445000.0,Café,Gastropub,Bakery,Deli / Bodega,Pizza Place,Convenience Store,Coffee Shop,Fish & Chips Shop,Garden Center,Gas Station
